# Hydrojug Phase 2 project: Demand vs Inventory (5-Month Forecasting Horizon)

## Goal
I need to match up what customers want (Forecast + Open Orders) against what is actually in stock over the next 5-month forecasting horizon.
I'm focusing on answering three big questions:
- What can be shipped with current and incoming inventory?
- Which UPCs require ordering now (keeping in mind the 90-day lead time)?
- Where might I need to push demand out to later months?

## Definitions (given)
- **Combined demand** = Forecast + Open Orders. When orders and forecasts overlap, I count the order but not both.
- **Available inventory** is ready to ship right now.
- **Committed** inventory is already baked into the On Hand (OH) numbers.
- **Pending Approval** and **Unallocated** don't touch inventory until someone manually changes it.
- **Backordered** items will auto-commit once inventory shows up.
- **Scope:** I'm limiting the analysis to UPCs that have actual inventory data.
- **Lead Time:** Assumed to be 90 days across the board.

## Preliminaries 1: confirm the workbook is accessible and has the expected sheets

Before working on this project, I'm making sure two things are true:

1) The Excel file is actually where I expect it (`data/Data Analysis Exercise_Phase II.xlsx`).
2) Python can open it and see the worksheet names.

This test catches issues early. Wrong file path, renamed file, incomplete download, etc. If this works, I know I can load each sheet into data tables without surprises downstream.


In [71]:
import pandas as pd

path = "data/Data Analysis Exercise_Phase II.xlsx"
xl = pd.ExcelFile(path)
xl.sheet_names


['Received Orders', 'Forecast Per Channel', 'Inventory']

## Preliminaries 2: confirm the notebook is using the intended Python environment

Jupyter notebooks run inside a selected "kernel" (Python environment). To avoid confusion, and make this reproducible for anyone checking my work, I'm printing:
- The Python executable path (so you can see which environment is actually running)
- The installed pandas version

This confirms I'm using the project environment where all the required packages are installed, not some random system Python that's missing dependencies.

In [72]:
import sys, pandas as pd
print("Python executable:", sys.executable)
print("Pandas version:", pd.__version__)


Python executable: c:\Users\dunnc\OneDrive\Desktop\hydrojug-phase2\.venv\Scripts\python.exe
Pandas version: 2.3.3


## Load the three source tables from Excel

The workbook gives me three pieces of the puzzle I need to combine:

- **Received Orders**: demand already in the system (open and committed quantities)
- **Forecast Per Channel**: planned demand by channel, broken down by month
- **Inventory**: current on-hand stock plus incoming supply and status buckets

Next, I'm loading each sheet into a dataframe and checking row counts to make sure the import actually worked.


In [73]:
import pandas as pd

path = r"data\Data Analysis Exercise_Phase II.xlsx"

received = pd.read_excel(path, sheet_name="Received Orders")
forecast  = pd.read_excel(path, sheet_name="Forecast Per Channel")
inventory = pd.read_excel(path, sheet_name="Inventory")

received.shape, forecast.shape, inventory.shape


((417, 6), (1011, 10), (297, 12))

## Inspect table structure (columns)

Before I join any tables or start running calculations, I'm inspecting the column names to identify:

- The item identifier (UPC)
- The channel identifier
- The quantity and date fields needed for demand and supply

This "data orientation" helps me get my bearings so the rest of the analysis is transparent and easy for you to audit.

In [74]:
for name, df in {"received": received, "forecast": forecast, "inventory": inventory}.items():
    print(f"\n{name.upper()} — rows={df.shape[0]}, cols={df.shape[1]}")
    print(df.columns.tolist())



RECEIVED — rows=417, cols=6
['customer', 'product_type', 'color', 'upc', 'SUM of qty_open', 'SUM of qty_committed']

FORECAST — rows=1011, cols=10
['forecast_node', 'product_type', 'color', 'upc', '2025-11', '2025-12', '2026-01', '2026-02', '2026-03', '2026-04']

INVENTORY — rows=297, cols=12
['Specification', 'Product', 'UPC', 'Color', 'Available', 'Committed', 'Unallocated', 'Backordered', 'Pending Approval', 'Incoming', 'Next Expected Date', 'On Hand']


## Inspect sample rows (spot formatting and data issues early)

I'm taking a look at a few rows from each table to confirm the following:

- Whether UPCs look like numbers or text
- Whether channel names are consistent
- Where the relevant quantity fields actually sit

I'd rather catch subtle formatting idiosyncrasies now than have it break my joins later.

In [75]:
display(received.head())
display(forecast.head())
display(inventory.head())


,customer,product_type,color,upc,SUM of qty_open,SUM of qty_committed
0,Channel 7,Coffee Traveler 20oz,Black,810140554956,79,79
1,Channel 7,Coffee Traveler 20oz,Brown Leopard,810140556165,76,0
2,Channel 7,Coffee Traveler 20oz,Dusty Rose,810140553379,79,79
3,Channel 7,Coffee Traveler 20oz,Rodeo,810140556233,83,83
4,Channel 7,Everyday Tumbler 20oz,Black,810140554987,144,144


,forecast_node,product_type,color,upc,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04
0,Channel 1,Coffee Traveler 20oz,Black,810140554956,120.0,118.0,88.0,118.0,118.0,128.0
1,Channel 1,Coffee Traveler 20oz,Brown Leopard,810140556165,71.0,50.0,49.0,29.0,29.0,32.0
2,Channel 1,Coffee Traveler 20oz,Camo,810140554857,71.0,50.0,50.0,30.0,30.0,32.0
3,Channel 1,Coffee Traveler 20oz,Cascade,810140554512,78.0,68.0,48.0,53.0,53.0,58.0
4,Channel 1,Coffee Traveler 20oz,Courtyard,810140553317,127.0,89.0,89.0,53.0,53.0,58.0


,Specification,Product,UPC,Color,Available,Committed,Unallocated,Backordered,Pending Approval,Incoming,Next Expected Date,On Hand
0,Core,Traveler 40oz,810084159064,Cream,27688,5398,1592,0,0,21092,2026-01-19 00:00:00,33086
1,Core,Traveler 40oz,810084159057,Black,15129,5923,1865,0,4,30485,2026-02-12 00:00:00,21052
2,Core,Traveler 40oz,810084159040,Pink Sand,11794,10730,0,0,1,29102,2025-03-17 00:00:00,22524
3,Core,Traveler 40oz,810084159088,Sage,10247,3562,0,0,184,12514,2026-02-12 00:00:00,13809
4,Core,Traveler 32oz,810084151341,Black,9549,5575,0,0,0,21992,2026-02-14 00:00:00,15124


## Normalize UPCs and limit scope to UPCs with inventory data

Everything here depends on UPCs matching across worksheets. I'll be making sure that Excel is storing UPCs consistently.

I'm creating a normalized UPC key (`upc_norm`) in each dataframe to handle this.

**The Rule:** I only care about UPCs that actually have inventory data.
So I'm filtering the forecast and orders down to just the UPCs present in the Inventory sheet.

In [76]:
def norm_upc(s: pd.Series) -> pd.Series:
    # Convert to string, remove trailing ".0" if Excel stored as a float, strip whitespace
    return (
        s.astype(str)
         .str.replace(r"\.0$", "", regex=True)
         .str.strip()
    )

received["upc_norm"]   = norm_upc(received["upc"])
forecast["upc_norm"]   = norm_upc(forecast["upc"])
inventory["upc_norm"]  = norm_upc(inventory["UPC"])

# Only assess UPCs that have inventory data
inv_upcs = set(inventory["upc_norm"].dropna())
received_in_scope = received[received["upc_norm"].isin(inv_upcs)].copy()
forecast_in_scope = forecast[forecast["upc_norm"].isin(inv_upcs)].copy()

received_in_scope.shape, forecast_in_scope.shape, inventory.shape


((208, 7), (760, 11), (297, 13))

## Analysis and a few assumptions

### Forecasting Horizon
The forecast sheet breaks demand into monthly blocks (like `2025-11`, `2025-12`, etc.).  
When I refer to the "next 5 months," I am defining the **forecasting horizon** as the first 5 months appearing in the dataset.

### Combined demand (given)
Open Orders and Forecast have overlapping elements. Since the company fulfills actual orders, not forecasted numbers, there's no risk of double-counting.

There's a catch though: open orders in `Received Orders` don't have ship dates attached. So I have to make an educated guess about which months they belong to.

**My approach (forecast-shaped allocation):**
- Take each UPC + Channel combo and spread its total open orders across months based on how the forecast is already distributed for that channel.
- Then figure out combined monthly demand like this:
  
$$
\text{CombinedDemand}_{m} = \max(\text{Forecast}_{m},\ \text{AllocatedOpenOrders}_{m})
$$

This way I avoid double-counting and keep everything lined up month by month.

### Supply definition
- I start with `On Hand` inventory (which already factors in committed stock).
- I add `Incoming` inventory based on when it's supposed to arrive (`Next Expected Date`).
- `Unallocated` and `Pending Approval` sit on the sidelines until someone manually changes their status. They don't reduce inventory automatically.
- Standard lead time runs 90 days (about 3 months).

## Identify the forecast month blocks programmatically

Forecast demand comes in monthly blocks labeled like `YYYY-MM`.
I'm detecting these columns programmatically so the analysis isn't dependent on hard-coded month names. This ensures the logic remains robust even if the time horizon shifts.

In [77]:
import re

# Detect forecast month columns like "2025-11"
month_cols = [
    c for c in forecast_in_scope.columns
    if isinstance(c, str) and re.fullmatch(r"\d{4}-\d{2}", c)
]

month_cols = sorted(month_cols)
month_cols


['2025-11', '2025-12', '2026-01', '2026-02', '2026-03', '2026-04']

## Select the "next 5 months" horizon for the exercise

Since the exercise specifically asked for the next five months, I'm defining the horizon as the first five forecast months found in the dataset.

This sets up a clear, reproducible window to compare:
- **Demand** (forecast + orders, ensuring no double counting)
vs
- **Supply** (on hand + incoming receipts during those same months)

In [78]:
HORIZON_MONTHS = month_cols[:5]
HORIZON_MONTHS


['2025-11', '2025-12', '2026-01', '2026-02', '2026-03']

## Ensure channel labels align between forecast and orders

The forecast is organized by **channel**, and orders also reference a channel (via the customer or channel field).
To combine these correctly, I'm standardizing the channel strings and checking for discrepancies.

If the mismatch list is empty, I can confidently join on `(UPC, Channel)` without needing a manual mapping table.

In [79]:
# Normalize channel names so joins work cleanly
received_in_scope["channel"] = received_in_scope["customer"].astype(str).str.strip()
forecast_in_scope["channel"] = forecast_in_scope["forecast_node"].astype(str).str.strip()

# Quick check: which channels appear in orders but not in forecast?
orders_channels = set(received_in_scope["channel"].unique())
fcst_channels = set(forecast_in_scope["channel"].unique())

sorted(orders_channels - fcst_channels)


[]

## Build a time-phased forecast table by UPC and channel

I'm aggregating the forecast into a clean structure:

- Index: `(upc_norm, channel)`
- Columns: the five months of the forecasting horizon
- Values: forecast units per month

This gives me a monthly demand curve per UPC per channel.

In [80]:
import pandas as pd

forecast_wide = (
    forecast_in_scope
    .groupby(["upc_norm", "channel"])[HORIZON_MONTHS]
    .sum()
    .fillna(0)
)

forecast_wide.head()


2025-11  2025-12  2026-01  2026-02  2026-03
upc_norm     channel                                               
810084150450 Channel 1   1718.0   1718.0   1714.0   1028.0   1028.0
             Channel 2   1805.0   2720.0   1396.0   1210.0   1190.0
             Channel 3    152.0    190.0    190.0    114.0    114.0
             Channel 6    945.0    945.0    943.0    566.0    566.0
             Channel 7      0.0      0.0     72.0    312.0    168.0

## Aggregate open and committed orders by UPC and channel

Orders represent demand I need to prioritize. The dataset gives me two types:

- **qty_open**: Not yet committed to inventory
- **qty_committed**: Already committed (but still demand that must ship)

I'm summing both to get the total order demand per `(UPC, Channel)`.

**Note:** The provided orders table doesn't actually include a requested ship date. Because of this missing piece, I'm making an explicit assumption to distribute these orders across the monthly forecasting horizon.

In [81]:
# Auto-detect the qty columns (handles "SUM of qty_open" naming)
open_col = next(c for c in received_in_scope.columns if "qty_open" in c)
comm_col = next(c for c in received_in_scope.columns if "qty_committed" in c)

orders_by_channel = (
    received_in_scope
    .groupby(["upc_norm", "channel"])[[open_col, comm_col]]
    .sum()
    .rename(columns={open_col: "qty_open", comm_col: "qty_committed"})
)

orders_by_channel["orders_total"] = orders_by_channel["qty_open"] + orders_by_channel["qty_committed"]
orders_by_channel.head()


qty_open  qty_committed  orders_total
upc_norm     channel                                         
810084150177 Channel 7       576              0           576
             Channel 8      2160              2          2162
810084150450 Channel 7       986              0           986
             Channel 8       979             38          1017
             Channel 9         4              4             8

## Compute combined demand without double counting (forecast vs orders)

**The Rule:** Combined demand corresponds to forecast plus open orders, but since they overlap, I need to fulfill the orders without double-counting the demand.

Because the orders in this dataset aren't month-dated, I'm making an explicit time-phasing assumption:

**Forecast-shaped allocation**
- I distribute each `(UPC, Channel)`'s total orders across the months proportional to that channel's forecast pattern.

Then I define the monthly combined demand as:

$$
\text{CombinedDemand}_m = \max(\text{Forecast}_m,\ \text{AllocatedOrders}_m)
$$

This satisfies the "no double counting" requirement while keeping the demand properly time-phased across the forecasting horizon.

In [82]:
import numpy as np

ORDER_ALLOCATION_METHOD = "forecast_shape"  # options: "forecast_shape", "all_in_first_month"

def allocate_orders_across_months(forecast_row: np.ndarray, orders_total: float) -> np.ndarray:
    """
    Returns a monthly vector for open orders.
    - forecast_shape: allocate orders proportional to forecast distribution
    - all_in_first_month: allocate all orders to first month (very conservative)
    """
    f = forecast_row.astype(float)
    o = float(orders_total)

    if o <= 0:
        return np.zeros_like(f)

    if ORDER_ALLOCATION_METHOD == "all_in_first_month":
        out = np.zeros_like(f)
        out[0] = o
        return out

    # Default: forecast-shaped allocation
    total_f = f.sum()
    if total_f > 0:
        return o * (f / total_f)

    # If there's no forecast shape to use, be conservative: put orders in first month
    out = np.zeros_like(f)
    out[0] = o
    return out

def combined_demand_monthly(forecast_row: np.ndarray, orders_total: float) -> np.ndarray:
    allocated_orders = allocate_orders_across_months(forecast_row, orders_total)
    return np.maximum(forecast_row.astype(float), allocated_orders)

# Join forecast with orders totals
fc_join = forecast_wide.join(orders_by_channel[["orders_total"]], how="left")
fc_join["orders_total"] = fc_join["orders_total"].fillna(0)

# Compute combined demand at UPC+Channel level
combined_by_channel = fc_join.apply(
    lambda row: pd.Series(
        combined_demand_monthly(row[HORIZON_MONTHS].to_numpy(), row["orders_total"]),
        index=HORIZON_MONTHS
    ),
    axis=1
)

combined_by_channel.head()


2025-11  2025-12      2026-01      2026-02  \
upc_norm     channel                                                 
810084150450 Channel 1   1718.0   1718.0  1714.000000  1028.000000   
             Channel 2   1805.0   2720.0  1396.000000  1210.000000   
             Channel 3    152.0    190.0   190.000000   114.000000   
             Channel 6    945.0    945.0   943.000000   566.000000   
             Channel 7      0.0      0.0   128.608696   557.304348   

                            2026-03  
upc_norm     channel                 
810084150450 Channel 1  1028.000000  
             Channel 2  1190.000000  
             Channel 3   114.000000  
             Channel 6   566.000000  
             Channel 7   300.086957

## Roll up from channel-level to UPC-level planning

Since inventory is managed at the UPC level, I'm rolling up the demand from channel-level to UPC-level.

This aggregation gives me:
- Combined monthly demand by UPC
- Baseline forecast monthly demand by UPC (for comparison)
- Total orders by UPC (for context)

This creates the consolidated demand view I need to compare against available and incoming supply.

In [83]:
combined_by_upc = combined_by_channel.groupby("upc_norm")[HORIZON_MONTHS].sum()

forecast_by_upc = forecast_wide.groupby("upc_norm")[HORIZON_MONTHS].sum()

orders_by_upc = orders_by_channel.groupby("upc_norm")[["qty_open", "qty_committed", "orders_total"]].sum()

combined_by_upc.head()


,2025-11,2025-12,2026-01,2026-02,2026-03
upc_norm,,,,,
810084150450,4620.000000,5573.000000,5267.608696,5063.304348,3486.086957
810084150566,1252.000000,1269.000000,1746.548387,1811.274194,1072.177419
810084150665,0.000000,0.000000,18.000000,82.000000,40.000000
810084150672,557.000000,730.000000,754.902243,937.381058,526.716699
810084150689,1401.888579,1297.626741,1523.993966,1895.398557,866.092156


## Build the supply timeline: On Hand inventory and scheduled receipts

Supply over the forecasting horizon consists of two main components:

1) **Starting supply:** `On Hand` from the Inventory table (which already includes committed inventory).
2) **Incoming supply:** `Incoming` quantities arriving on the `Next Expected Date`.

I'm converting the expected receipt dates into monthly blocks so I can directly compare incoming supply against demand, month by month.

In [84]:
inv = inventory.copy()

inv["next_expected_date"] = pd.to_datetime(inv["Next Expected Date"], errors="coerce")
inv["incoming_month"] = inv["next_expected_date"].dt.strftime("%Y-%m")

inv_by_upc = inv.set_index("upc_norm")

on_hand = inv_by_upc["On Hand"].fillna(0)
incoming = inv_by_upc["Incoming"].fillna(0)

# Incoming receipts bucketed into our horizon months (force numeric + fill NaNs)
receipts_by_upc = (
    inv.pivot_table(index="upc_norm", columns="incoming_month", values="Incoming", aggfunc="sum")
      .reindex(columns=HORIZON_MONTHS, fill_value=0)
      .fillna(0)
)

receipts_by_upc.head()


incoming_month,2025-11,2025-12,2026-01,2026-02,2026-03
upc_norm,,,,,
50000162611,0,0,0.0,0.0,0.0
50000162711,0,0,0.0,0.0,0.0
810084150177,0,0,0.0,6966.0,0.0
810084150450,0,0,0.0,13979.0,0.0
810084150566,0,0,0.0,10012.0,0.0


## Data quality check: how much incoming supply is inside the forecasting horizon?

To ensure I don't overstate near-term supply, I'm quantifying the following:

- Total incoming supply listed in the inventory table
- Incoming receipts that actually land inside the 5-month forecasting horizon
- Incoming quantities that have no readable expected date

This helps me interpret shortages correctly and flags data issues before they become a problem.

In [85]:
incoming_total = inv_by_upc["Incoming"].fillna(0).sum()
incoming_in_horizon = receipts_by_upc.sum().sum()

missing_date_incoming = inv.loc[
    (inv["Incoming"].fillna(0) > 0) & (inv["next_expected_date"].isna()),
    "Incoming"
].sum()

incoming_total, incoming_in_horizon, missing_date_incoming


(np.int64(622524), np.float64(478185.0), np.int64(27080))

## Investigate receipts outside the forecasting horizon (early vs late)

Incoming receipts that fall outside the forecasting horizon still matter for my interpretation:

- **Early (before the forecasting horizon):** These might already be available by the start (or could indicate past-due receipts).
- **Late (after the forecasting horizon):** These can't help me meet demand within the 5-month window.

I'm listing and labeling these out-of-horizon receipts to highlight timing risks and potential date-quality issues.

In [86]:
first_month = HORIZON_MONTHS[0]
last_month = HORIZON_MONTHS[-1]

out_of_horizon = inv.loc[
    (inv["Incoming"].fillna(0) > 0) &
    (inv["incoming_month"].notna()) &
    (~inv["incoming_month"].isin(HORIZON_MONTHS)),
    ["UPC", "incoming_month", "Incoming", "Next Expected Date"]
].copy()

out_of_horizon["timing_vs_horizon"] = np.where(
    out_of_horizon["incoming_month"] < first_month, "EARLY (before horizon)",
    np.where(out_of_horizon["incoming_month"] > last_month, "LATE (after horizon)", "OTHER")
)

out_of_horizon.sort_values("Incoming", ascending=False).head(15)


,UPC,incoming_month,Incoming,Next Expected Date,timing_vs_horizon
2,810084159040,2025-03,29102,2025-03-17 00:00:00,EARLY (before horizon)
16,810084153895,2025-03,13410,2025-03-17 00:00:00,EARLY (before horizon)
5,810084154076,2025-03,10562,2025-03-17 00:00:00,EARLY (before horizon)
9,810084151563,2025-03,8132,2025-03-17 00:00:00,EARLY (before horizon)
118,840494401491,2026-05,5702,2026-05-30 00:00:00,LATE (after horizon)
205,840494429044,2026-04,5162,2026-04-29 00:00:00,LATE (after horizon)
141,840494433638,2026-05,4500,2026-05-30 00:00:00,LATE (after horizon)
206,840494429051,2026-04,4475,2026-04-30 00:00:00,LATE (after horizon)
207,840494429068,2026-05,4082,2026-05-01 00:00:00,LATE (after horizon)
208,840494429075,2026-04,3812,2026-04-29 00:00:00,LATE (after horizon)


## Simulate inventory position month-by-month (per UPC)

I'm now netting the time-phased supply against the demand to simulate the flow.

For each UPC and month, I calculate the end-of-month inventory position as:

$$
\text{Position}_m = \text{StartOnHand} + \sum_{t \le m}\text{Receipts}_t - \sum_{t \le m}\text{Demand}_t
$$

**How I interpret this:**
- **Position ≥ 0** → Demand is covered on time.
- **Position < 0** → I have a backlog at that point (late or unfilled demand).

In [87]:
analysis_upcs = sorted(set(forecast_in_scope["upc_norm"]).union(set(received_in_scope["upc_norm"])))

demand = combined_by_upc.reindex(analysis_upcs).fillna(0)
receipts = receipts_by_upc.reindex(analysis_upcs).fillna(0)
start_inventory = on_hand.reindex(analysis_upcs).fillna(0)

# End-of-month inventory position each month:
# position[m] = start + cumulative_receipts_through_m - cumulative_demand_through_m
inv_position = (
    start_inventory.to_numpy()[:, None]
    + receipts.cumsum(axis=1).to_numpy()
    - demand.cumsum(axis=1).to_numpy()
)

inv_position_df = pd.DataFrame(inv_position, index=analysis_upcs, columns=HORIZON_MONTHS)

inv_position_df.head()


,2025-11,2025-12,2026-01,2026-02,2026-03
810084150177,2.0,2.0,2.000000,6968.000000,6968.0
810084150450,-4578.0,-10151.0,-15418.608696,-6502.913043,-9989.0
810084150566,-1220.0,-2489.0,-4235.548387,3965.177419,2893.0
810084150580,2753.0,2753.0,2753.000000,2753.000000,2753.0
810084150610,565.0,565.0,565.000000,565.000000,565.0


## Headline comparison: total demand vs total supply in the 5-month window

I'm computing the high-level numbers:

- **Total combined demand** across the five months
- **Total supply** available in the same window (On Hand + in-horizon incoming)

I know that even if the totals indicate a surplus, SKU-level shortages can still happen because inventory isn't interchangeable across UPCs, and receipts might arrive after the demand hits.

In [88]:
total_combined_demand_5mo = demand.sum().sum()
total_on_hand = start_inventory.sum()
total_incoming_5mo = receipts.sum().sum()
total_supply_5mo = total_on_hand + total_incoming_5mo

total_combined_demand_5mo, total_supply_5mo


(np.float64(697623.0), np.float64(877988.0))

## Quantify service risk: backlog vs true shortage

I'm computing two complementary metrics to assess risk:

- **Peak backlog:** The maximum unmet demand at any point in the forecasting horizon (a timing/service issue).
- **End-of-horizon shortage:** Unmet demand still remaining after all receipts within the forecasting horizon have arrived (a true supply gap within the window).

This allows me to distinguish between the mental labels "temporarily late" and "I simply don't have enough units by the end of the forecasting horizon."

In [89]:
end_pos = inv_position_df[HORIZON_MONTHS[-1]]
end_shortage = (-end_pos).clip(lower=0)

peak_backlog = (-inv_position_df).clip(lower=0).max(axis=1)  # max units unmet at any point

n_upcs_any_backlog = int((peak_backlog > 0).sum())
n_upcs_end_short = int((end_shortage > 0).sum())

n_upcs_any_backlog, n_upcs_end_short, float(end_shortage.sum())

(75, 31, 96077.0)

## Incorporate the 90-day lead time into ordering decisions

Given a standard lead time of 90 days (aprox. 3 months), I know that a new purchase order placed today won't resolve shortages in the first 3 months of the forecasting horizon.

I'm approximating this constraint by:
- Treating month 4 of the forecasting horizon as the earliest point new supply could arrive
- Computing the additional quantity required per UPC to keep the inventory position non-negative from that month onward

I'm also flagging the backlog **before** this earliest arrival month. These are units I can't fix by ordering under the given lead time assumption; they will likely require rescheduling demand or some other operational intervention.

In [90]:
LEAD_TIME_MONTHS = 3  # 90 days

if len(HORIZON_MONTHS) > LEAD_TIME_MONTHS:
    arrival_month = HORIZON_MONTHS[LEAD_TIME_MONTHS]
    post_months = HORIZON_MONTHS[LEAD_TIME_MONTHS:]  # months you *could* influence with a new PO arriving at arrival_month
    pre_months = HORIZON_MONTHS[:LEAD_TIME_MONTHS]

    # If we add supply at arrival_month, how much is needed to keep inventory >= 0 from arrival_month onward?
    min_post = inv_position_df[post_months].min(axis=1)
    order_qty_if_arrives = (-min_post).clip(lower=0)

    backlog_before_arrival = (-inv_position_df[pre_months]).clip(lower=0).max(axis=1)

    arrival_month, float(order_qty_if_arrives.sum()), float(backlog_before_arrival.sum())

else:
    print("Horizon is too short to apply a 3-month lead time rule.")


## Build a UPC-level decision table (actionable results)

To make the results actionable and easy to grade, I'm compiling a single table per UPC that includes:

- Starting On Hand
- Incoming within the forecasting horizon
- Combined demand within the forecasting horizon
- End-of-horizon position and shortage
- Peak backlog (timing risk)
- Lead-time-feasible order quantities (if applicable)

I'm sorting this by shortage to highlight exactly which UPCs need my attention first.


In [91]:
product = inv_by_upc["Product"] if "Product" in inv_by_upc.columns else pd.Series(index=inv_by_upc.index, dtype="object")
color = inv_by_upc["Color"] if "Color" in inv_by_upc.columns else pd.Series(index=inv_by_upc.index, dtype="object")

summary = pd.DataFrame({
    "Product": product.reindex(analysis_upcs),
    "Color": color.reindex(analysis_upcs),
    "Start_On_Hand": start_inventory,
    "Incoming_in_Horizon": receipts.sum(axis=1),
    "Combined_Demand_5mo": demand.sum(axis=1),
    "End_Inv_Position": end_pos,
    "End_Shortage": end_shortage,
    "Peak_Backlog": peak_backlog,
})

# Add lead-time-based order qty if computed
if len(HORIZON_MONTHS) > LEAD_TIME_MONTHS:
    summary[f"Order_Qty_if_Arrives_{arrival_month}"] = order_qty_if_arrives
    summary[f"Backlog_Before_{arrival_month}"] = backlog_before_arrival

summary.sort_values("End_Shortage", ascending=False).head(15)


,Product,Color,Start_On_Hand,Incoming_in_Horizon,Combined_Demand_5mo,End_Inv_Position,End_Shortage,Peak_Backlog,Order_Qty_if_Arrives_2026-02,Backlog_Before_2026-02
810084159040,Traveler 40oz,Pink Sand,22524,0.0,42967.0,-20443.0,20443.0,20443.000000,20443.000000,3138.070743
810084151136,Traveler 32oz,Cream,10472,0.0,23153.0,-12681.0,12681.0,12681.000000,12681.000000,2478.224033
810084150450,Sport Kids 14oz,Dinosaur,42,13979.0,24010.0,-9989.0,9989.0,15418.608696,9989.000000,15418.608696
810084154076,Traveler 40oz,Wildrose,14842,0.0,24810.0,-9968.0,9968.0,9968.000000,9968.000000,0.000000
810084153895,Traveler 40oz,Camo,8243,0.0,15373.0,-7130.0,7130.0,7130.000000,7130.000000,0.000000
850010870250,Squeeze Bottle 27oz,Mauve,29,0.0,4749.0,-4720.0,4720.0,4720.000000,4720.000000,3084.275362
840494401491,Everyday Tumbler 14oz,Brown Leopard,9,0.0,4254.0,-4245.0,4245.0,4245.000000,4245.000000,2012.000000
810084159200,Traveler 20oz,Rodeo,2806,0.0,5692.0,-2886.0,2886.0,2886.000000,2886.000000,0.000000
810084151563,Traveler 32oz,Sage,10395,0.0,12964.0,-2569.0,2569.0,2569.000000,2569.000000,0.000000
810140552525,Traveler 20oz,Sage,999,0.0,3214.0,-2215.0,2215.0,2215.000000,2215.000000,692.947619


## Identify timing-only UPCs (candidates to move demand instead of ordering)

Some UPCs go negative in the middle of the forecasting horizon but recover by the end once receipts arrive.
I view these as **timing-only** problems:

- I might not need to order more units overall.
- Instead, I might need to push demand later (backorder/reschedule), expedite supply, or change operational status.

I'm flagging timing-only UPCs where:
- Peak backlog > 0
- End shortage = 0

In [92]:
timing_only = summary[(summary["Peak_Backlog"] > 0) & (summary["End_Shortage"] == 0)].copy()
timing_only.shape


(44, 10)

## Prioritize timing-only issues by severity

Finally, I'm ranking the timing-only UPCs by peak backlog. This allows me to focus on the largest service risks first and identify exactly where rescheduling will have the biggest impact.

In [93]:
timing_only.sort_values("Peak_Backlog", ascending=False).head(20)


,Product,Color,Start_On_Hand,Incoming_in_Horizon,Combined_Demand_5mo,End_Inv_Position,End_Shortage,Peak_Backlog,Order_Qty_if_Arrives_2026-02,Backlog_Before_2026-02
810084150771,Traveler 32oz,Pink Sand,5825,28984.0,27350.0,7459.0,0.0,13030.952507,0.000000,13030.952507
810140552518,Traveler 20oz,Pink Sand,609,8723.0,9247.0,85.0,0.0,5649.121495,0.000000,5649.121495
810084159552,Sport 32oz,Rodeo,4303,11732.0,10268.0,5767.0,0.0,4470.028018,4470.028018,1126.325758
810140552464,Traveler 20oz,Cream,1423,10178.0,9725.0,1876.0,0.0,4399.871429,0.000000,4399.871429
810084150566,Sport Kids 14oz,Daisy Checkers,32,10012.0,7151.0,2893.0,0.0,4235.548387,0.000000,4235.548387
810140552501,Sport 32oz,Pink Sand,8098,8014.0,13495.0,2617.0,0.0,3800.316667,3800.316667,249.183333
810084154113,Traveler 32oz,Wildrose,4130,15844.0,15115.0,4859.0,0.0,3549.770436,0.000000,3549.770436
810140556141,Everyday Tumbler 20oz,Brown Leopard,6,6905.0,3757.0,3154.0,0.0,2853.880309,2853.880309,1701.613900
810084159057,Traveler 40oz,Black,21052,30485.0,40558.0,10979.0,0.0,2482.514589,0.000000,2482.514589
840494401439,Everyday Tumbler 14oz,Dusty Rose,83,3602.0,3059.0,626.0,0.0,2330.000000,2330.000000,1590.000000


## Conclusion and Notes on Methodology

### Summary of Results
In this analysis, I was able to synthesize the disparate data sources—Forecast, Received Orders, and Inventory into a unified, time-phased view of the next 5-month forecasting horizon. By normalizing UPCs and resolving channel-level discrepancies, I was able to model the inventory flow for every active SKU.

The final output is a prioritized decision table that distinguishes between:
1.  **True Shortages:** Where I simply do not have enough stock coming in (requiring new orders, subject to the 90-day lead time).
2.  **Timing/Backlog Risks:** Where stock is coming, but arriving too late to prevent a stockout (requiring expediting or demand management).

### Mathematical Basis
The formulas employed in this notebook rely on standard operations management logic:

1.  **Demand Consumption (The "Greater-Of" Rule):**
    My calculation for combined demand, $\text{CombinedDemand}_m = \max(\text{Forecast}_m, \text{Orders}_m)$, is derived from standard consumption logic used in Master Production Scheduling (MPS). This ensures that actual booked orders "consume" the forecast, preventing the over-inflation of demand signals (double counting).
    * *Reference:* Silver, E. A., Pyke, D. F., & Thomas, D. J. (2016). *Inventory and Production Management in Supply Chains*. CRC Press. (See chapters on Demand Management and MPS).

2.  **Inventory Position (Flow Balance):**
    The month-over-month simulation follows the fundamental Law of Inventory Conservation (Flow Balance Equation):
    $$I_t = I_{t-1} + \text{Supply}_t - \text{Demand}_t$$
    This recursive formula allows for the detection of negative inventory states (backlogs) at any specific node in the timeline, rather than just assessing the end-state.

### Technical Implementation & Tools
To complete this project within the allotted timeline, I employed principles of **prompt engineering** to expedite the generation of the Python/Pandas syntax. While I utilized AI to accelerate the coding of standard dataframe operations (joins, grouping, and aggregations), the analytical architecture, logic definition, assumption setting (such as the forecast-shaped allocation), and final validation of the results are entirely my own work.

### Data confidentiality note (source file not published)

Because I was not given warning instructions on whether the provided dataset is proprietary, I intentionally prevented the raw Excel data from being published to the repository. Specifically, before publishing I:

- Added a `.gitignore` rule to exclude Excel files in the `data/` directory (e.g., `data/*.xlsx`).
- Removed the Excel file from Git tracking (so it is not included in the repository contents going forward).
- Re-exported the notebook results to a self-contained HTML report (`docs/index.html`) so the analysis can be reviewed without distributing the raw source file.

As a result, anyone viewing the repository will be able to inspect the methodology and outputs via the HTML report, but will not receive the underlying raw dataset through the repo.
